In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data cleaning

In [2]:
df=pd.read_csv('lc_loan.csv/lc_loan.csv',low_memory=False)

In [3]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    

In [5]:
df.shape

(887379, 74)

In [6]:
#We look to see which columns have missing values
missing = pd.concat([df.isnull().sum(), 100 * df.isnull().mean()], axis=1)
missing.columns=['count','%']
missing[missing["count"]>0].sort_values(by='count',ascending=False)

,count,%
dti_joint,886870,99.942640
annual_inc_joint,886868,99.942415
verification_status_joint,886868,99.942415
il_util,868762,97.902024
mths_since_rcnt_il,866569,97.654892
total_cu_tl,866007,97.591559
inq_fi,866007,97.591559
all_util,866007,97.591559
max_bal_bc,866007,97.591559
open_rv_24m,866007,97.591559


Some of the most commonly missing values are related to joint applications. We can see how many there are.

In [7]:
df[df.application_type=="JOINT"].shape

(511, 74)

There are only 511 joint applications. While we may want to come back to these later, for a first run through we can get rid of them. Doing so we can get rid of the application type field. We will also take out the member ID field for modeling.

In [8]:
X=df[df.application_type!="JOINT"]
X=X.drop(columns=['id','member_id','application_type'])

We can now get rid of any columns involving the joint applicant

In [9]:
X=X.drop(columns=['dti_joint','annual_inc_joint','verification_status_joint'])

We can get the 2016-2017 data (to be used as testing) and start doing the same thing

In [10]:
df2=pd.read_csv("lc_2016_2017.csv/lc_2016_2017.csv",low_memory=False)

In [11]:
X_test=df2[df2.application_type!="JOINT"]
X_test=X_test.drop(columns=['id','member_id','application_type','dti_joint','annual_inc_joint','verification_status_joint'])

In [12]:
#Separate the target variable and remove it from the features
y=X.loan_status
y_test=X.loan_status
X.drop(columns=['loan_status'],inplace=True)
X_test.drop(columns=['loan_status'],inplace=True)

In [13]:
#We look to see which columns in the restricted data have missing values still
missing = pd.concat([X.isnull().sum(), 100 * X.isnull().mean()], axis=1)
missing.columns=['count','%']
missing[missing["count"]>0].sort_values(by='count',ascending=False)

,count,%
il_util,868392,97.916714
mths_since_rcnt_il,866220,97.671807
inq_last_12m,865659,97.608551
open_acc_6m,865659,97.608551
total_cu_tl,865659,97.608551
inq_fi,865659,97.608551
all_util,865659,97.608551
max_bal_bc,865659,97.608551
open_rv_24m,865659,97.608551
open_rv_12m,865659,97.608551


For the features with many missing values we want to see what's actually there

In [14]:
X.il_util.value_counts()

0.0      138
100.0    132
87.0      68
75.0      66
86.0      64
        ... 
118.8      1
113.0      1
136.3      1
133.2      1
116.9      1
Name: il_util, Length: 1271, dtype: int64

In [15]:
X.il_util.unique()

array([  nan,  49.3,  88.8, ..., 133.1,  12.9, 116.9])

In [16]:
X.il_util.describe()

count    18476.000000
mean        71.519788
std         23.034225
min          0.000000
25%         58.600000
50%         75.000000
75%         87.600000
max        223.300000
Name: il_util, dtype: float64

We're missing a lot of data here. There's quite a range here. While this potentially could cause issues we could impute the median of 75 here. We will impute the same to the test data 

In [17]:
X.il_util.fillna(75,inplace=True)

In [18]:
X_test.il_util.fillna(75,inplace=True)

In [19]:
X.mths_since_rcnt_il.value_counts()

4.0      1220
3.0      1179
7.0      1073
5.0      1041
6.0       980
         ... 
338.0       1
230.0       1
238.0       1
275.0       1
250.0       1
Name: mths_since_rcnt_il, Length: 200, dtype: int64

In [20]:
X.mths_since_rcnt_il.unique()

array([ nan,  28.,  11.,  47.,  13.,   8.,  21., 338.,  54.,  73.,  14.,
        18.,  23.,  42.,  19.,  10.,  27.,   2.,  76.,   7.,   6.,   5.,
        15.,   9.,  62.,  35.,   3.,  17.,  45.,  16.,  38.,  24.,  25.,
        12.,   1.,  43.,  91.,  55.,  52.,  37.,  50.,  79.,  64.,  22.,
        36.,  58.,  49.,  26.,   4.,  61.,  59.,  46., 141.,  32.,  53.,
        56., 100., 103.,  33.,  20.,  51.,  63.,  40., 118.,  98.,  31.,
       275., 121.,  29., 124.,  34.,  89.,  41.,   0., 145.,  44.,  77.,
        82.,  80.,  39.,  30.,  71., 111.,  95., 129.,  68., 168., 119.,
       115., 151.,  72.,  67., 104.,  74.,  93.,  57., 238., 148.,  87.,
       170., 230.,  66., 158., 107., 101.,  90., 150.,  78., 114., 120.,
       133., 136.,  60., 137.,  85., 131., 130.,  75.,  48., 109.,  81.,
       117., 102., 105., 113.,  88.,  83.,  84., 147.,  65.,  69.,  99.,
        86., 110., 152., 288., 135., 108., 149., 140., 112.,  94.,  97.,
       123., 134., 116.,  92., 138., 169., 139., 12

In [21]:
X.mths_since_rcnt_il.describe()

count    20648.000000
mean        20.922801
std         27.233241
min          0.000000
25%          6.000000
50%         12.000000
75%         23.000000
max        363.000000
Name: mths_since_rcnt_il, dtype: float64

This is months since most recent installment account was opened. Missing values are likely people that have no old one. The easiest way to make sure it's considered separate is to simply impute a very large number. For now we can use infinite. Since we likely will use decision trees here, that will be a reasonable way to separate them.

This will mean that trying to look at statistics on these fields will not make much sense.

In [22]:
X.mths_since_rcnt_il.fillna(np.inf,inplace=True)
X_test.mths_since_rcnt_il.fillna(np.inf,inplace=True)

In [23]:
X.inq_last_12m.value_counts()

 1.0     5011
 0.0     3958
 2.0     3728
 3.0     2576
 4.0     1681
-4.0     1365
 5.0      987
 6.0      664
 7.0      424
 8.0      272
 9.0      143
 10.0     112
 11.0      73
 12.0      57
 13.0      55
 14.0      27
 15.0      26
 16.0      12
 17.0       8
 19.0       8
 20.0       7
 18.0       5
 21.0       3
 25.0       2
 30.0       1
 26.0       1
 22.0       1
 32.0       1
 24.0       1
Name: inq_last_12m, dtype: int64

It seems reasonable to impute 0 for missing ones since the reasonable conclusion of no data on inquiries in the last 12 months is that there were zero. We have a fwe entries that are -4. That obviously isn't reasonable since you can't have negative inquiries. We can try to see what those say for inquiries in the last 6 months.

In [24]:
X[X.inq_last_12m==-4].inq_last_6mths.value_counts()

0.0    1365
Name: inq_last_6mths, dtype: int64

All 1365 had none in the last 6 months. So we will replace those with 0 as well. We will replace all missing and negative entries in the training and test data with 0

In [25]:
X.inq_last_12m=X.inq_last_12m.replace(-4,0)
X.inq_last_12m.fillna(0,inplace=True)

In [26]:
X.inq_last_12m.value_counts()

0.0     870982
1.0       5011
2.0       3728
3.0       2576
4.0       1681
5.0        987
6.0        664
7.0        424
8.0        272
9.0        143
10.0       112
11.0        73
12.0        57
13.0        55
14.0        27
15.0        26
16.0        12
19.0         8
17.0         8
20.0         7
18.0         5
21.0         3
25.0         2
26.0         1
30.0         1
22.0         1
32.0         1
24.0         1
Name: inq_last_12m, dtype: int64

In [27]:
X_test.inq_last_12m.fillna(0,inplace=True)
#There were no negative values in the test data so we didn't have to worry about that.

In [32]:
missing[missing['count']>0].index

Index(['emp_title', 'emp_length', 'annual_inc', 'desc', 'title', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_util',
       'total_acc', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl',
       'inq_last_12m'],
      dtype='object')

In [34]:
X.annual_inc.describe()

count    8.868640e+05
mean     7.503702e+04
std      6.471049e+04
min      1.896000e+03
25%      4.500000e+04
50%      6.500000e+04
75%      9.000000e+04
max      9.500000e+06
Name: annual_inc, dtype: float64

We only had a few missing annual incomes. We will simply use the median of the test data (65000) for those

In [36]:
X.annual_inc.fillna(65000,inplace=True)
X_test.annual_inc.fillna(65000,inplace=True)

In [37]:
X.emp_title.value_counts()

Teacher                                   13453
Manager                                   11233
Registered Nurse                           5521
Owner                                      5375
RN                                         5352
                                          ...  
The Sams Clinic                               1
Annalect Group                                1
Coast Composites                              1
Antelope Valley Union High School Dist        1
Manager Hotel Operations Oasis                1
Name: emp_title, Length: 299159, dtype: int64

We will replace missing values of title as 'other'

In [39]:
X.emp_title.fillna('other',inplace=True)
X_test.emp_title.fillna('other',inplace=True)

In [40]:
X.emp_length.value_counts()

10+ years    291403
2 years       78833
< 1 year      70559
3 years       69994
1 year        57064
5 years       55686
4 years       52496
7 years       44576
8 years       43930
6 years       42928
9 years       34635
Name: emp_length, dtype: int64

In [41]:
X.emp_length.unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

We want to try to find roughly the median employement length. This is only going to be rough because we have categories of < 1 year and 10+ years
Roughly half the values are at 6 years or below and half at 7 years and above. For now we will try using 7 years

In [65]:
X.emp_length.fillna('7 years',inplace=True)
X_test.emp_length.fillna('7 years',inplace=True)

In [66]:
X.emp_length.value_counts()

10+ years    291403
7 years       89340
2 years       78833
< 1 year      70559
3 years       69994
1 year        57064
5 years       55686
4 years       52496
8 years       43930
6 years       42928
9 years       34635
Name: emp_length, dtype: int64